In [4]:
import hydra
import torch
from tqdm.notebook import tqdm
import time
from lib.utils.config import load_config
from lib.optimizer.framework import NeuralOptimizer, NeuralOptimizer2
from lib.data.loader import load_intrinsics
from lib.data.loader import load_intrinsics
from lib.rasterizer import Rasterizer
from lib.renderer.renderer import Renderer
from lib.renderer.camera import Camera
from lib.tracker.timer import TimeTracker
from lib.utils.progress import reset_progress, close_progress


def path_to_abblation(path):
    return "_".join(path.split("/")[-3].split("_")[1:])


def eval_iterations(optimizer, datamodule, backward: bool=False, strategy: str="forward-mode"):
    tracker = TimeTracker()
    optimizer.max_iters = 1
    optimizer.max_optims = 1
    optimizer.optimizer.strategy=strategy

    # loop
    for batch in datamodule.val_dataloader():
        batch = optimizer.transfer_batch_to_device(batch, "cuda", 0)
        tracker.start("step")
        out = optimizer(batch)
        if backward:
            out["params"] = batch["init_params"]
            loss_info = optimizer.compute_loss(batch=batch, out=out)
            loss_info["loss"].backward()
        tracker.stop("step")

    return tracker


def load_flame_renderer():
    # instanciate similar to training
    cfg = load_config("train", ["data=synthetic"])
    K = load_intrinsics(data_dir=cfg.data.intrinsics_dir, return_tensor="pt")
    camera = Camera(
        K=K,
        width=cfg.data.width,
        height=cfg.data.height,
        near=cfg.data.near,
        far=cfg.data.far,
        scale=cfg.data.scale,
    )
    rasterizer = Rasterizer(width=camera.width, height=camera.height)
    renderer = Renderer(rasterizer=rasterizer, camera=camera)
    flame = hydra.utils.instantiate(cfg.model)
    return flame, renderer


def load_neural_optimizer(flame, renderer, path):
    cfg = load_config("train", ["data=synthetic"])
    correspondence = hydra.utils.instantiate(cfg.correspondence)
    weighting = hydra.utils.instantiate(cfg.weighting)
    residuals = hydra.utils.instantiate(cfg.residuals)
    regularize = hydra.utils.instantiate(cfg.regularize)
    neural_optimizer = NeuralOptimizer.load_from_checkpoint(
        path,
        renderer=renderer,
        flame=flame,
        correspondence=correspondence,
        regularize=regularize,
        residuals=residuals,
        weighting=weighting,
    )
    return neural_optimizer


# setup the datamodule
def load_datamodule(renderer, start_frame, end_frame):
    cfg = load_config("train", ["data=synthetic"])
    datamodule = hydra.utils.instantiate(
        cfg.data,
        renderer=renderer,
        val_dataset=dict(
            start_frame=start_frame,
            end_frame=end_frame,
        ),
    )
    datamodule.setup("fit")
    return datamodule

In [5]:
# settings
start_frame = 10
end_frame = 14

# checkpoints
ours = "/home/borth/GuidedResearch/logs/2024-10-06/06-55-55_abblation_ours_ckpt/checkpoints/last.ckpt"

# loadings
times = {}
p_losses = {}
v_losses = {}
flame, renderer = load_flame_renderer()
datamodule = load_datamodule(renderer, start_frame, end_frame)

Creating GL context for cuda device 0
Successfully initialized EGL version 1.5
Successfully initialized OpenGL version 4.6.0 NVIDIA 535.183.01


In [7]:
optimizer = load_neural_optimizer(flame, renderer, ours)
tracker = eval_iterations(optimizer, datamodule, backward=False, strategy="reverse-mode")
print(tracker.print_summary())


+-----------+---------+--------+---------+--------+--------+-------+----------+
| task_name |    mean |    min |     max | median |    std | steps |    total |
+-----------+---------+--------+---------+--------+--------+-------+----------+
|      step | 186.111 | 137.74 | 232.682 | 187.98 | 29.918 |    32 | 5955.549 |
+-----------+---------+--------+---------+--------+--------+-------+----------+


In [ ]:
optimizer = load_neural_optimizer(flame, renderer, ours)
tracker = eval_iterations(optimizer, datamodule, backward=False, strategy="reverse-mode")
print(tracker.print_summary())

In [8]:
optimizer = load_neural_optimizer(flame, renderer, ours)
tracker = eval_iterations(optimizer, datamodule, backward=False, strategy="forward-mode")
print(tracker.print_summary())


+-----------+------+--------+---------+--------+--------+-------+----------+
| task_name | mean |    min |     max | median |    std | steps |    total |
+-----------+------+--------+---------+--------+--------+-------+----------+
|      step | 44.8 | 27.267 | 104.044 | 40.142 | 16.205 |    32 | 1433.612 |
+-----------+------+--------+---------+--------+--------+-------+----------+


In [1]:
import pandas as pd

df = pd.read_csv("/home/borth/GuidedResearch/temp/wandb_export_2024-09-02T14_08_05.306+02_00.csv")
df["iters"] = df.Name.str.split("_").str[2:].str.join("_")
df = df.groupby("iters").mean("train/init_loss").reset_index()
df = df.drop(columns=["Runtime"])

icp1 = {
    "method": "icp1",
    "train_flame": df[df["iters"] == "1_params_p2p"]["train/init_loss"].iloc[0],
    "val_flame": df[df["iters"] == "1_params_p2p"]["val/init_loss"].iloc[0],
    "train_p2p": df[df["iters"] == "1_params_p2p"]["train/init_point2point"].iloc[0],
    "val_p2p": df[df["iters"] == "1_params_p2p"]["val/init_point2point"].iloc[0],
    "FPS": 54.94, 
}
icp3 = {
    "method": "icp3",
    "train_flame": df[df["iters"] == "3_params_p2p"]["train/init_loss"].iloc[0],
    "val_flame": df[df["iters"] == "3_params_p2p"]["val/init_loss"].iloc[0],
    "train_p2p": df[df["iters"] == "3_params_p2p"]["train/init_point2point"].iloc[0],
    "val_p2p": df[df["iters"] == "3_params_p2p"]["val/init_point2point"].iloc[0],
    "FPS": 18.55, 
}
icp10 = {
    "method": "icp10",
    "train_flame": df[df["iters"] == "10_params_p2p"]["train/init_loss"].iloc[0],
    "val_flame": df[df["iters"] == "10_params_p2p"]["val/init_loss"].iloc[0],
    "train_p2p": df[df["iters"] == "10_params_p2p"]["train/init_point2point"].iloc[0],
    "val_p2p": df[df["iters"] == "10_params_p2p"]["val/init_point2point"].iloc[0],
    "FPS": 5.52, 
}
ours1 = {
    "method": "ours1",
    "train_flame": df[df["iters"] == "1_params_p2p"]["train/loss"].iloc[0],
    "val_flame": df[df["iters"] == "1_params_p2p"]["val/loss"].iloc[0],
    "train_p2p": df[df["iters"] == "1_params_p2p"]["train/point2point"].iloc[0],
    "val_p2p": df[df["iters"] == "1_params_p2p"]["val/point2point"].iloc[0],
    "FPS": 54.34, 
}
ours3 = {
    "method": "ours3",
    "train_flame": df[df["iters"] == "3_params_p2p"]["train/loss"].iloc[0],
    "val_flame": df[df["iters"] == "3_params_p2p"]["val/loss"].iloc[0],
    "train_p2p": df[df["iters"] == "3_params_p2p"]["train/point2point"].iloc[0],
    "val_p2p": df[df["iters"] == "3_params_p2p"]["val/point2point"].iloc[0],
    "FPS": 18.26, 
}
ours3_params = {
    "method": "ours3_params",
    "train_flame": df[df["iters"] == "3_params"]["train/loss"].iloc[0],
    "val_flame": df[df["iters"] == "3_params"]["val/loss"].iloc[0],
    "train_p2p": df[df["iters"] == "3_params"]["train/point2point"].iloc[0],
    "val_p2p": df[df["iters"] == "3_params"]["val/point2point"].iloc[0],
    "FPS": 18.26, 
}
ours3_p2p = {
    "method": "ours3_p2p",
    "train_flame": df[df["iters"] == "3_p2p"]["train/loss"].iloc[0],
    "val_flame": df[df["iters"] == "3_p2p"]["val/loss"].iloc[0],
    "train_p2p": df[df["iters"] == "3_p2p"]["train/point2point"].iloc[0],
    "val_p2p": df[df["iters"] == "3_p2p"]["val/point2point"].iloc[0],
    "FPS": 18.26, 
}

eval_df = pd.DataFrame([icp1, icp3, icp10, ours1, ours3, ours3_params, ours3_p2p])
eval_df

,method,train_flame,val_flame,train_p2p,val_p2p,FPS
0,icp1,1.641144,0.687063,3.964142,1.911708,54.94
1,icp3,1.568537,0.690532,2.538077,1.673656,18.55
2,icp10,0.818773,0.647058,1.749077,1.568222,5.52
3,ours1,0.923439,1.033893,2.915562,2.452145,54.34
4,ours3,0.571321,0.924628,2.229952,1.937396,18.26
5,ours3_params,0.554304,0.885214,2.583386,1.790254,18.26
6,ours3_p2p,1.568502,0.694314,2.538101,1.673804,18.26


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

reverse_no_grad = {
    "method": "reverse_no_grad",
    "16x": 25.015,
    "8x": 82.966,
    "4x": 384.760,
    "2x": 0.0,
    "1x": 0.0,
}
reverse_grad = {
    "method": "reverse_grad",
    "16x": 27.698,
    "8x": 82.966,
    "4x": 477.149,
    "2x": 0.0,
    "1x": 0.0,
}
reverse_diff = {
    "method": "reverse_diff",
    "16x": 166.238,
    "8x": 0.0,
    "4x": 0.0,
    "2x": 0.0,
    "1x": 0.0,
}
forward_no_grad = {
    "method": "forward_no_grad",
    "16x": 11.612,
    "8x": 11.916,
    "4x": 14.858,
    "2x": 27.307,
    "1x": 75.879,
}
forward_grad = {
    "method": "forward_grad",
    "16x": 12.409,
    "8x": 12.446,
    "4x": 15.419,
    "2x": 27.481,
    "1x": 76.292,
}
forward_diff = {
    "method": "forward_diff",
    "16x": 45.590,
    "8x": 46.995,
    "4x": 54.543,
    "2x": 91.013,
    "1x": 243.967,
}
df = pd.DataFrame(
    [
        forward_no_grad,
        forward_grad,
        forward_diff,
        reverse_no_grad,
        reverse_grad,
        reverse_diff,
    ]
)

# Extracting method names and the x multipliers
methods = df["method"]
x_multipliers = ["16x", "8x", "4x", "2x", "1x"]
# Extracting the values for each multiplier
values = df[x_multipliers].values

# Set up the color map
colors = plt.get_cmap("Set2", len(df["method"]))

# Set up the figure and axes
fig, ax = plt.subplots(figsize=(7, 3))

# Number of bars per group
n_bars = len(df["method"])

# Set the width of the bars
bar_width = 0.15

# Set the spacing between groups
group_spacing = 0.2  # Adjust this value for more or less space between groups

# Set the positions of the bars with added spacing
bar_positions = np.arange(len(df.columns) - 1) * (n_bars * bar_width + group_spacing)


# Plotting each method as a separate bar
for i, (method, color) in enumerate(zip(df["method"], colors.colors)):
    ax.bar(
        bar_positions + i * bar_width, values[i], bar_width, label=method, color=color
    )

# Adding x-ticks
ax.set_xticks(bar_positions + bar_width * (n_bars - 1) / 2)
ax.set_xticklabels(["440", "1794", "7138", "28636", "115398"]) 

# Adding labels and title
ax.set_xlabel("Residuals (count)")
ax.set_ylabel("Time (ms)")
ax.set_title("Jacobian Performance Comparison by Method")
# Adding a red line at 30ms
ax.axhline(y=30, color='red', linestyle='--')

# Adding a legend
ax.legend(loc="upper left")

# Adding grid for better readability
ax.grid(True, linestyle="--", alpha=0.7)

# Show the plot
plt.tight_layout()

plt.show()